In [33]:
from twitch import TwitchClient
import tweepy
import pandas as pd
import requests
import datetime
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret,twitch_client_id)

client = TwitchClient(client_id=twitch_client_id)
#channel = client.channels.get_by_id(44322889)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
currtime = datetime.datetime.now().strftime('%m/%d/%y')



In [2]:
client.games.get_top()

[{'game': {'name': 'Fortnite',
   'popularity': 246668,
   'id': 33214,
   'giantbomb_id': 37030,
   'box': {'large': 'https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-272x380.jpg',
    'medium': 'https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-136x190.jpg',
    'small': 'https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-52x72.jpg',
    'template': 'https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-{width}x{height}.jpg'},
   'logo': {'large': 'https://static-cdn.jtvnw.net/ttv-logoart/Fortnite-240x144.jpg',
    'medium': 'https://static-cdn.jtvnw.net/ttv-logoart/Fortnite-120x72.jpg',
    'small': 'https://static-cdn.jtvnw.net/ttv-logoart/Fortnite-60x36.jpg',
    'template': 'https://static-cdn.jtvnw.net/ttv-logoart/Fortnite-{width}x{height}.jpg'},
   'links': {},
   'localized_name': 'Fortnite',
   'locale': 'en-us'},
  'viewers': 248870,
  'channels': 13525},
 {'game': {'name': 'Counter-Strike: Global Offensive',
   'popularity': 70162,
   'id': 32399,
   'giantbomb_id': 36113,
   'box'

In [22]:
print(client.games.get_top()[0]['viewers'])

249521


In [3]:
#client.channels.get_by_id(15350)
#client.channels.get_videos(44322889)
#client.users.get_by_id(18)

In [23]:
games = len(client.games.get_top())
top_games = []
popularity = []
viewers = []
channels = []
for i in range(games):
    top_games.append(client.games.get_top()[i]['game']['name'])
    popularity.append(client.games.get_top()[i]['game']['popularity'])
    viewers.append(client.games.get_top()[i]['viewers'])
    channels.append(client.games.get_top()[i]['channels'])



df = pd.DataFrame({
    'Game' : top_games,
    'Popularity' : popularity,
    'Views': viewers,
    'Channels': channels
}
)

df


,Game,Popularity,Views,Channels
0,Fortnite,242800,249521,13344
1,League of Legends,65874,66496,2228
2,Counter-Strike: Global Offensive,66041,65175,943
3,Marvel's Spider-Man,64212,64055,766
4,World of Warcraft,59200,58977,3187
5,Tom Clancy's Rainbow Six: Siege,46098,45879,1425
6,IRL,48120,45626,900
7,Overwatch,38731,37904,1492
8,Destiny 2,36309,36284,2571
9,Always On,25026,25051,19


In [39]:
response = requests.get('https://api.duckduckgo.com/?q=DuckDuckGo&format=json').json()
response

{'AbstractText': "DuckDuckGo is an Internet search engine that emphasizes protecting searchers' privacy and avoiding the filter bubble of personalized search results. DuckDuckGo distinguishes itself from other search engines by not profiling its users and by deliberately showing all users the same search results for a given search term, and emphasizes returning the best results, rather than the most results, generating those results from over 400 individual sources, including crowdsourced sites such as Wikipedia, and other search engines like Bing, Yahoo!, and Yandex.",
 'Results': [{'FirstURL': 'https://duckduckgo.com/',
   'Icon': {'Width': 16,
    'Height': 16,
    'URL': 'https://duckduckgo.com/i/duckduckgo.com.ico'},
   'Text': 'Official site',
   'Result': '<a href="https://duckduckgo.com/"><b>Official site</b></a><a href="https://duckduckgo.com/"></a>'}],
 'Redirect': '',
 'ImageWidth': 340,
 'ImageHeight': 270,
 'Type': 'A',
 'Infobox': {'content': [{'sort_order': '1',
    'lab

In [ ]:
https://contextualwebsearch.com/api/Search/WebSearchAPIWithPagination?q=[q]&pageNumber=[pageNumber]&pageSize=[pageSize]&autoCorrect=true